# From Mantid to Scipp
## Data types
### Workspaces

| Mantid | Scipp |
| ---| --- |
| `Workspace2D` | `DataArray` |
| `EventWorkspace` | `DataArray` |
| `WorkspaceSingleValue` | `DataArray` |
| `MDHistoWorkspace` | `DataArray` |
| `MDEventWorkspace` | not supported |
| `TableWorkspace` | `Dataset` |
| `WorkspaceGroup` | `Dataset` (aligned dimensions), otherwise Python `list` or `dict` |

#### Notes
- In many cases it may be desirable to use `Dataset` instead of `DataArray`.
- Scipp (other rather conversion to scipp) is currently still incomplete and does not carry over all information from a workspace.

### Other

| Mantid | Scipp |
| ---| --- |
| `DetectorInfo` | `Dataset` |

## Algorithms

### Notes
- In **Mantid** a Python variable referencing **a workspace is** under the hood **a global variable**.
  Unless specific otherwise the variable name is the name of the workspace in the [AnalysisDataService](https://docs.mantidproject.org/nightly/concepts/AnalysisDataService.html).
  For marginally more clarity, the examples in the following therefore use the string-based syntax for specifying output workspaces.
  *In scipp there is no such limitation and everything behaves just like normal variables in Python.*
- Unless stated otherwise, the following code examples assume datasets or data arrays have `Dim.Tof` for what Mantid calls "X" and `Dim.Spectrum` why Mantid calls "Y" or "spectrum axis".
- There is not strict 1:1 equivalence between Mantid workspaces and functionality in scipp.
  The examples below give the most common examples, but in many cases exceptions apply and detailed behavior may differ.
  If in doubt, consult the Mantid algorithm documentation and the scipp documentation.

In [ ]:
import mantid.simpleapi as mantid
import scipp as sc
from scipp import Dim
import numpy as np

### Generic algorithms

#### CloneWorkspace

In [ ]:
mantid.CloneWorkspace(InputWorkspace='data', OutputWorkspace='copy')

Equivalent in scipp:

In [ ]:
copy = data.copy()

#### DeleteWorkspace

In [ ]:
mantid.DeleteWorkspace(Workspace='data')

Equivalent in scipp:

In [ ]:
del data

#### ExtractSingleSpectrum

In [ ]:
mantid.ExtractSingleSpectrum(
    InputWorkspace='data',
    OutputWorkspace='spec',
    WorkspaceIndex=7)

Equivalent in scipp:

In [ ]:
spec = data[Dim.Spectrum, 7]

If an actual *copy* is required use:

In [ ]:
spec = data[Dim.Spectrum, 7].copy()

#### ExtractSpectra / CropWorkspace

In [ ]:
mantid.ExtractSpectra(
    InputWorkspace='data',
    OutputWorkspace='spectra',
    StartWorkspaceIndex=7,
    EndWorkspaceIndex=88)

Equivalent in scipp:

In [ ]:
spectra = data[Dim.Spectrum, 7:89]

If an actual *copy* is required use:

In [ ]:
spectra = data[Dim.Spectrum, 7:89].copy()

#### Transpose

In [ ]:
mantid.Transpose(InputWorkspace='data', OutputWorkspace='data')

Equivalent in scipp:
Transposing is *implicit* and automatic based on dimension labels and not required for any of the common operations, including plotting.

#### AppendSpectra

In [ ]:
mantid.AppendSpectra(
    InputWorkspace1='data1',
    InputWorkspace2='data2',
    OutputWorkspace='data')

Equivalent in scipp:

In [ ]:
data = sc.concatenate(data1, data2, Dim.Spectrum)

#### ConjoinXRuns

In [ ]:
mantid.ConjoinXRuns(
    InputWorkspaces='data1,data2',
    OutputWorkspace='data')

Equivalent in scipp:

In [ ]:
data = sc.concatenate(data1, data2, Dim.Tof)

#### ConjoinSpectra

In [ ]:
mantid.ConjoinSpectra(
    InputWorkspaces='ws1,ws2',
    OutWorkspace='spec_vs_temperature'
    WorkspaceIndex=7,
    LabelUsing="temperature",
    LabelValue="Mean")

Equivalent in scipp:

In [ ]:
data = sc.concatenate(data1, data2, Dim.Temperature)
sample_temperature = data.attrs['sample'].value['temperature']
data.coords[Dim.Temperature] = sc.mean(sample_temperature, Dim.Time)
spec_vs_temperature = data[Dim.Temperature, 7]

#### GroupWorkspaces

In [ ]:
mantid.GroupWorkspaces(
    InputWorkspaces='ws1,ws2',
    OutputWorkspace='data')

Equivalent in scipp:

In [ ]:
data = sc.Dataset({
    'data1':data_array1,
    'data2':data_array2})

This requires aligned dimensions (matching coordinates) in all input arrays.
Use a Python `dict` or `list` for grouping unaligned data.

#### Rebin  `Workspace2D` into `Workspace2D`

In [ ]:
mantid.Rebin(
    InputWorkspace='histo',
    OutputWorkspace='histo',
    Params='0,100,20000')

Equivalent in scipp:

In [ ]:
edges = sc.Variable(
    [Dim.Tof],
    values=np.arange(0.0, 20000.0, 100.0),
    unit=sc.units.us)
histo = sc.rebin(histo, Dim.Tof, edges)

#### Rebin  `EventWorkspace` preserving events

In [ ]:
mantid.Rebin(
    InputWorkspace='events',
    OutputWorkspace='events',
    Params='0,100,20000',
    PreserveEvents=True)

Equivalent in scipp:

In [ ]:
edges = sc.Variable(
    [Dim.Tof],
    values=np.arange(0.0, 20000.0, 100.0),
    unit=sc.units.us)
events.coords[Dim.Tof] = edges

#### Rebin  `EventWorkspace` into `Workspace2D`

In [ ]:
mantid.Rebin(
    InputWorkspace='events',
    OutputWorkspace='histo',
    Params='0,100,20000',
    PreserveEvents=False)

Equivalent in scipp:

In [ ]:
edges = sc.Variable(
    [Dim.Tof],
    values=np.arange(0.0, 20000.0, 100.0),
    unit=sc.units.us)
histo = sc.histogram(events, edges)

#### Scale (multiplication)

In [ ]:
mantid.Scale(
    InputWorkspace='data',
    OutputWorkspace='data', 
    Factor=7.5,
    Operation="Multiply")

Equivalent in scipp:

In [ ]:
data *= 7.5

#### Scale (addition)

In [ ]:
mantid.Scale(
    InputWorkspace='data',
    OutputWorkspace='data', 
    Factor=7.5,
    Operation="Add")

Equivalent in scipp:

In [ ]:
data += 7.5

If the data is not dimensionless, the correct unit must be specified:

In [ ]:
data += 7.5 * sc.units.us

#### ScaleX

In [ ]:
mantid.ScaleX(
    InputWorkspace='data',
    OutputWorkspace='data', 
    Factor=7.5,
    Operation="Multiply")

Equivalent in scipp:

In [ ]:
data.coords[Dim.Tof] *= 7.5

### Neutron-scattering specific algorithms
#### ConvertUnits

In [ ]:
ConvertUnits(InputWorkspace='tof', OutputWorkspace='dspacing', Target='dSpacing')

Equivalent in scipp:

In [ ]:
dspacing = sc.neutron.convert(tof, Dim.Tof, Dim.DSpacing)